##CHUẨN BỊ

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

##DETECTION: Viola John algorithm

In [2]:
# Tải về bộ phân lớp Haar đã được huấn luyện để phát hiện khuôn mặt
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

##RECOGNITION (face + face emotion): HOG + SVM

In [3]:
import os
from skimage.feature import hog
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

In [4]:
"""Xây dựng mô hình phân lớp SVM
Input: dir: đường dẫn đến thư mục train (chứa các ảnh để huấn luyện)
Output: model: mô hình SVM đã được huấn luyện
"""
def train_model(dir):
    features = []
    labels = []

    # Duyệt qua từng thư mục trong dir
    for label_dir in os.listdir(dir):
        label_path = dir + "/" + label_dir
        
        # Đảm bảo đường dẫn là một thư mục
        if os.path.isdir(label_path):
            # Nhãn là tên thư mục
            label = label_dir
            
            # Duyệt qua từng file trong thư mục label
            for file_name in os.listdir(label_path):
                file_path = label_path + "/" + file_name
                
                # Đọc và chuyển ảnh sang ảnh xám .
                img = cv2.imread(file_path)
                if img is None:
                    print('Lỗi '+file_path)
                    continue
                else:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # Phát hiện khuôn mặt
                bboxes = face_cascade.detectMultiScale(image=img)
                
                # Nhận dạng khuôn mặt
                for (column, row, width, height) in bboxes:
                    # Cắt và scale vùng ảnh chứa khuôn mặt
                    face = img[row:row+height, column:column+width]
                    face = cv2.resize(face, (128, 128))
                    
                    # Trích xuất đặc trưng HOG
                    feat = hog(face, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
                    
                    # Thêm đặc trưng và nhãn vào mảng
                    features.append(feat)
                    labels.append(label)
                
    # Chuyển đổi sang mảng numpy
    features = np.array(features)
    labels = np.array(labels)
    
    # Xây dựng mô hình SVM
    model = SVC()
    # model.fit(features, labels)
    # return model

    # Định nghĩa các giá trị tham số cần điều chỉnh
    param_grid = {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
        'gamma': [0.1, 1, 10]
    }

    # Thực hiện Grid Search để điều chỉnh tham số
    grid_search = GridSearchCV(model, param_grid, cv=5)
    grid_search.fit(features, labels)

    # Lấy ra mô hình tốt nhất đã điều chỉnh
    best_model = grid_search.best_estimator_
    return best_model

In [5]:
# Tải mô hình từ file (ko cần phải train lại)
from joblib import load
try:
    model_fr = load('../trained_model/fr_model.joblib')
    model_fer = load('../trained_model/fer_model.joblib')
    print('Load pretrained model from file successfully ~')
except:
    # Huấn luyện mô hình SVM để nhận dạng khuôn mặt
    print('Train face recognition model...')
    # model_fr = train_model("../Data/train/fr")
    # Huấn luyện mô hình SVM để nhận dạng cảm xúc
    print('Train face emotion recognition model...')
    model_fer = train_model("../Data/train/fer")

    # Lưu lại mô hình
    from joblib import dump
    try:
        path = '../trained_model/'
        if not os.path.exists(path):
            os.makedirs(path)
        # dump(model_fr, path + 'fr_model.joblib')
        dump(model_fer, path + 'fer_model.joblib')
        print('Save model successfully~')
    except:
        print('Fail to save the models!')

Load pretrained model from file successfully ~


In [6]:
"""Sử dụng mô hình phân lớp để nhận dạng khuôn mặt và cảm xúc
Input:
    data_dir: đường dẫn đến thư mục Data
    valid_file: đường dẫn đến file cần nhận dạng,
    img: ảnh xám của file cần nhận dạng,
    bboxes: các bounding box khuôn mặt được phát hiện,
    model_fr: mô hình để nhận dạng khuôn mặt,
    model_fer: mô hình để nhận dạng cảm xúc
Output: Ghi file kết quả nhận dạng vào thư mục Data/output
"""
def recognize(valid_file, img, bboxes, model_fr, model_fer):
    # Khởi tạo ảnh output bằng ảnh gốc ban đầu
    output_img = cv2.imread(valid_file)
    
    # Duyệt qua từng bounding box khuôn mặt được phát hiện
    for (x, y, w, h) in bboxes:
        # Cắt và scale khuôn mặt
        face_roi = img[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))

        # Trích đặc trưng 
        features = hog(face_roi, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        features = np.array(features).reshape(1, -1)

        # Dự đoán nhãn
        fr_predicted_label = model_fr.predict(features)[0]
        fer_predicted_label = model_fer.predict(features)[0]
        
        # Ghi kết quả lên ảnh
        cv2.rectangle(output_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(output_img, fr_predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2)
        cv2.putText(output_img, fer_predicted_label, (x, y + h + 15), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
    return output_img

In [7]:
"""Lưu lại các ảnh kết quả nhận dạng
Input:
    data_dir: đường dẫn đến thư mục Data
    output_img: ảnh kết quả
Output: Ghi file ảnh kết quả vào thư mục Data/output
"""
def write_result_img(data_dir, output_img): 
    # Ghi file output
    path = data_dir + "/output"
    if not os.path.exists(path):
        os.makedirs(path)

    # Đếm số lượng file hiện có trong thư mục
    count = 0
    for _, _, files in os.walk(path):
        count += len(files)
        
    # Tên file tiếp theo là số lượng file hiện tại + 1 
    count += 1
    path += "/" + str(count)+".jpg"
    check = cv2.imwrite(path, output_img)
    if not check:
        print("Fail to save file!", path)

In [9]:
"""Chạy thử mô hình trên tập ảnh valid sau khi đã huấn luyện"""
def test_model():
    data_dir="../Data"
    valid_dir = "../Data/valid/fr"
    # Duyệt qua từng thư mục cần kiểm thử
    for sub_dir in os.listdir(valid_dir):
        sub_path = valid_dir + "/" + sub_dir
        
        # Đảm bảo đường dẫn là một thư mục
        if os.path.isdir(sub_path):
            # Duyệt qua từng file ảnh trong thư mục
            for file in os.listdir(sub_path):
                file = sub_path + "/" + file
                # Đọc và chuyển ảnh sang ảnh xám 
                img = cv2.imread(file)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                # Phát hiện khuôn mặt
                bboxes = face_cascade.detectMultiScale(image=img, scaleFactor=1.3, minNeighbors=5)
                # Nhận diện khuôn mặt và cảm xúc
                result_img = recognize(file, img, bboxes, model_fr, model_fer)
                write_result_img(data_dir, result_img)
test_model()

In [13]:
"""Xử lý phát hiện, nhận dạng trên mỗi frame hình
Input: frame hình cần xử lý
Output: frame hình kết quả
"""
def process_frame(frame):
    #Đọc và chuyển ảnh sang ảnh xám 
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Phát hiện khuôn mặt
    bboxes = face_cascade.detectMultiScale(image=img, scaleFactor=1.3, minNeighbors=5)
    
    # Nhận diện khuôn mặt và cảm xúc
    # Duyệt qua từng bounding box khuôn mặt được phát hiện
    for (x, y, w, h) in bboxes:
        # Cắt và scale khuôn mặt
        face_roi = img[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (128, 128))

        # Trích đặc trưng 
        features = hog(face_roi, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2), visualize=False)
        features = np.array(features).reshape(1, -1)

        # Dự đoán nhãn
        fr_predicted_label = model_fr.predict(features)[0]
        fer_predicted_label = model_fer.predict(features)[0]
        
        # Ghi kết quả lên ảnh
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, fr_predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 2)
        cv2.putText(frame, fer_predicted_label, (x, y + h + 15), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 2)
    return frame

In [14]:
"""Thực hiện chạy mô hình đã được huấn luyện và test kết quả trên video"""

# Tên file video
video_file = '../Data/video1.mp4'
video = cv2.VideoCapture(video_file)

while video.isOpened():
    ret, frame = video.read()
    processed_frame = process_frame(frame)
    cv2.imshow('Video', processed_frame)
    cv2.setWindowProperty("Video", cv2.WND_PROP_AUTOSIZE, cv2.WINDOW_AUTOSIZE)

    # Đợi nhận sự kiện thoát cửa sổ khi người dùng nhấp vào nút X
    cv2.waitKey(1)
    if cv2.getWindowProperty("Video", cv2.WND_PROP_VISIBLE) < 1:
        break

video.release()
cv2.destroyAllWindows()

In [ ]:
# Lưu giữ đường cong cảm xúc (từng ID gương mặt theo thời gian)
# Điểm danh (đếm số lượng ID khác nhau theo thời gian)


In [ ]:
# import matplotlib.pyplot as plt

# # Lưu kết quả
# def save_emotion_results(emotion_predictions, output_file):
#     with open(output_file, 'w') as file:
#         for prediction in emotion_predictions:
#             time, emotion = prediction
#             file.write(f"{time}\t{emotion}\n")
#     print("Kết quả đã được lưu thành công.")

# # Xây dựng đường cong cảm xúc
# def plot_emotion_curve(emotion_predictions):
#     times = [prediction[0] for prediction in emotion_predictions]
#     emotions = [prediction[1] for prediction in emotion_predictions]
    
#     plt.plot(times, emotions)
#     plt.xlabel("Thời gian")
#     plt.ylabel("Cảm xúc")
#     plt.title("Đường cong cảm xúc theo thời gian")
#     plt.show()

# # Gọi các hàm
# emotion_predictions = [(0.1, 'happy'), (0.5, 'sad'), (1.2, 'angry'), ...]  # Thay bằng danh sách kết quả thực tế
# output_file = "emotion_results.txt"  # Thay bằng tên tập tin mà bạn muốn lưu kết quả

# save_emotion_results(emotion_predictions, output_file)
# plot_emotion_curve(emotion_predictions)
